import os
from Bio import SeqIO
import pandas as pd
import numpy as np
from statsmodels.sandbox.stats.multicomp import multipletests
import sys
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import dill

In [ ]:
os.environ['R_HOME'] = "/home/cfriedline/R3/lib64/R/"
import rpy2.ipython
import rpy2.robjects as robjects
%load_ext rpy2.ipython
r = robjects.r
ri2py = robjects.conversion.ri2py

In [ ]:
%%R
library(topGO)
library(qvalue)

In [ ]:
cd ~/g/projects/black_spruce_new/

In [ ]:
def shell(cmd):
    from subprocess import Popen, PIPE
    p = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE)
    stdout, stderr = p.communicate()
    return [x for x in stdout.split("\n") if x != '']#, stderr.split("\n")

In [ ]:
samples = ["P40C", "P40N", "P32C", "P32N"]

In [ ]:
for s in samples:
    fa = "%s.fa" % s
    ql = "%s.qual" % s
    for f in [fa, ql]:
        shell("cp ../black_spruce/$f .")

In [ ]:
fa_files = shell("ls *.fa")

In [ ]:
fa_files

##Run seqclean

In [ ]:
os.environ['PATH'] = "/home/cfriedline/g/src/blast-2.2.26/bin:%s" % os.environ['PATH']

In [ ]:
shell("which blastall")

In [ ]:
for f in fa_files:
    print f
    shell("~/g/src/seqclean-x86_64/seqclean %s \
    -v /home/cfriedline/g/src/univec_9/UniVec \
    -s ~/g/projects/Escherichia_coli_K_12_substr__DH10B_uid58979/NC_010473.fna" % f)

In [ ]:
clean_fa = shell("ls *.clean | grep -v all")

In [ ]:
clean_fa

##Combine into single fasta file

In [ ]:
with open("all.fa.clean", "w") as o:
    for f in clean_fa:
        sample = f.split(".")[0]
        for rec in SeqIO.parse(f, "fasta"):
            rec.id = "%s_%s" % (sample, rec.id)
            SeqIO.write(rec, o, "fasta")

In [ ]:
shell("grep -c '>' *.fa")

In [ ]:
shell("grep -c '>' *.clean")

##Input into Sequin for futher vector trimming

* select only strong/moderate
* trim 
* add moltype=mRNA
* add topology=linear
* export nucleotide fasta as `all.fa.clean.sequin`

##Remove sequences < 100nt

In [ ]:
with open("all.fa.clean.sequin.trim", "w") as o:
    for rec in SeqIO.parse("all.fa.clean.sequin", "fasta"):
        if len(rec) >= 100:
            SeqIO.write(rec, o, "fasta")

In [ ]:
shell("grep -c '>' *.sequin")

In [ ]:
shell("grep -c '>' *.trim")

## Run iAssembler

```bash
~/g/src/iAssembler-v1.3.2.x64/iAssembler.pl -a 20 -b 20 -i all.fa.clean.sequin.trim
```

##Blastx against uniprot (see uniprot.ipynb)

```bash
/home/cfriedline/gpfs/src/ncbi-blast-2.2.30+/bin/blastx \
-db ../black_spruce/uniprot_sprot_plants.fasta \
-query all.fa.clean.sequin.trim_output/unigene_seq.fasta \
-out all.fa.clean.sequin.trim_uniprot_sprot.xml \
-outfmt 5 \
-num_alignments 10 \
-evalue 1e-5 \
-num_threads 20
```

In [ ]:
shell("grep -c '>' all.fa.clean.sequin.trim_output/unigene_seq.fasta")

##Compute counts per tissue

In [ ]:
counts = {}
for line in open("all.fa.clean.sequin.trim_output/contig_member"):
    data = line.strip().split("\t")
    if not data[0] in counts:
        counts[data[0]] = {}
    for seq in data[1:]:
        tissue = seq.split("_")[0]
        if not tissue in counts[data[0]]:
            counts[data[0]][tissue] = 1
        else:
            counts[data[0]][tissue] += 1

In [ ]:
counts = pd.DataFrame(counts).T

In [ ]:
counts = counts.fillna(0)

In [ ]:
counts.head()

In [ ]:
count_file = "all.fa.clean.sequin.trim_output/contig_member.counts"

In [ ]:
counts.to_csv(count_file, sep="\t", header=True, index=True)

In [ ]:
counts = pd.read_csv(count_file, sep="\t", header=0, index_col=0)

In [ ]:
counts.head()

In [ ]:
def combine(row):
    return pd.Series([None, (row.P32C+row.P40C), (row.P32N+row.P40N)])
combined = counts.apply(combine, axis=1)
combined.columns = ["Descr", "C", "N"]
combined.index.name = "UNIQID"

In [ ]:
combined.head()

In [ ]:
totals = pd.DataFrame(columns=combined.columns)
totals.ix['UNIQID',:] = combined.apply(np.sum)
totals.ix['UNIQID','Descr'] = 'Descr'
totals

In [ ]:
df = pd.concat([totals, combined])

In [ ]:
df.Descr[1:] = df.index[1:]

In [ ]:
df.head()

In [ ]:
new_index = ["UNIQID"]
new_index.extend([int(x.replace("UN", "")) for x in df.index[1:]])

In [ ]:
df.index = new_index

In [ ]:
df.head()

In [ ]:
def convert_to_int(col):
    try:
        return col.astype(int)
    except:
        return col
df = df.apply(convert_to_int)
        
df.to_csv("ideg6_counts.txt", sep="\t", header=False, index=True, float_format="%.0f")

## Use IDEG6 web tool to calculate differentially expressed genes
http://telethon.bio.unipd.it/bioinfo/IDEG6_form/

bonferronni = 3.723008e-05

In [ ]:
bonferronni_alpha = 3.723008e-05

In [ ]:
dge_file = "ideg6_results.txt"

In [ ]:
results = pd.read_csv(dge_file, sep="\t", header=0, index_col=0)
results.columns = [x.replace(".", "") for x in results.columns]
results.columns = [x.replace("-", "_") for x in results.columns]
results.columns = [x.strip() for x in results.columns]
results = results.ix[:,:-1] #drop extra column at the end

In [ ]:
results.head()

In [ ]:
stat_cols = [u'AC1_2', u'Fisher1_2', u'Chi2x21_2', u'R', u'Chi']
qvalue_cols = [u'Fisher1_2', u'Chi2x21_2', u'R', u'Chi']

In [ ]:
def fdr_bh(pvals):
    return multipletests(pvals, method="fdr_bh")[1]

In [ ]:
def q_value(pvals):
    p = robjects.FloatVector(pvals)
    robjects.globalenv['p'] = p
    vals = r('qvalue(p)')
    return pd.Series(ri2py(vals.rx('pvalues')[0]))
qvalue_results = results[qvalue_cols].apply(q_value)
qvalue_results.columns = ["%s_q" % x for x in qvalue_results.columns]

In [ ]:
fdr_results = results[stat_cols].apply(fdr_bh)
fdr_results.columns = ["%s_fdr" % x for x in fdr_results.columns]

In [ ]:
results_df = results.join(fdr_results).join(qvalue_results)

In [ ]:
sns.set_context("talk")
X = sorted(results_df.Chi)
plt.step(X, np.arange(len(X)))
plt.show()

In [ ]:
results_df.Description = [x.strip() for x in results_df.Description]

In [ ]:
results_df[['Chi','Chi_fdr','Chi_q']][0:10]

In [ ]:
fdr_cols = [u'AC1_2_fdr', u'Fisher1_2_fdr', u'Chi2x21_2_fdr', u'R_fdr', u'Chi_fdr',
           'Fisher1_2_q', 'Chi2x21_2_q', 'R_q', 'Chi_q']

In [ ]:
fdr_res = pd.DataFrame(index=['total','sig'])
for col in fdr_cols:
    d = results_df[col]
    fdr_res[col] = [len(d), len(d[d<0.05])]
fdr_res.T

In [ ]:
stat_res = pd.DataFrame(index=['total','p<0.05', 'p<bonferroni'])
for col in stat_cols:
    d = results_df[col]
    stat_res[col] = [len(d), len(d[d<0.05]), len(d[d<bonferronni_alpha])]
stat_res.T

In [ ]:
len(results_df[results_df.Chi < 0.05]), len(results_df[results_df.Chi_fdr < 0.05])

In [ ]:
go_file = "topGO_full_blast2go_export_20150626_1451.txt"

In [ ]:
go = pd.read_csv(go_file, sep="\t", header=None, index_col=0, names=["go"])
go.index = [x.strip() for x in go.index]

In [ ]:
go.head()

In [ ]:
counts_go = counts.join(go)

In [ ]:
len(counts_go)

In [ ]:
results_df.index = [x.strip() for x in results_df.Description]

In [ ]:
results_df.head()

In [ ]:
len(results_df[results_df.Chi_fdr<0.05])

In [ ]:
full = counts_go.join(results_df)

In [ ]:
len(full)

In [ ]:
full_with_go = full.ix[full.go.dropna().index]

In [ ]:
len(full_with_go)

In [ ]:
sig = full[(full.Chi_fdr < 0.05)]

In [ ]:
len(sig)

In [ ]:
sig.head()

In [ ]:
sig_with_go = sig.ix[sig.go.dropna().index]

In [ ]:
len(sig_with_go)

In [ ]:
dge_N = sig_with_go[sig_with_go.Lib2 > sig_with_go.Lib1]
dge_C = sig_with_go[sig_with_go.Lib2 < sig_with_go.Lib1]

In [ ]:
def get_num_terms(x):
    return len(x.split(","))

print dge_C.go.apply(get_num_terms).describe()
print dge_N.go.apply(get_num_terms).describe()

In [ ]:
dge_N.ix[:,0:5].to_csv("dge_N.csv")
dge_C.ix[:,0:5].to_csv("dge_C.csv")

In [ ]:
len(sig_with_go), len(dge_C), len(dge_N)

In [ ]:
with open("dge_needle_names.txt", "w") as o:
    for name in dge_N.index.tolist():
        o.write("%s\n" % name)
        
with open("dge_cambium_names.txt", "w") as o:
    for name in dge_C.index.tolist():
        o.write("%s\n" % name)

In [ ]:
full_with_go['go'].to_csv("go_mappings.txt", sep="\t", header=False, index=True)

In [ ]:
len(full_with_go)

In [ ]:
shell("wc -l go_mappings.txt")

In [ ]:
%%R
rm(list=ls())

In [ ]:
robjects.globalenv['full_with_go'] = robjects.DataFrame(full_with_go)
robjects.globalenv['sig_with_go'] = robjects.DataFrame(sig_with_go)
robjects.globalenv['dge_C'] = robjects.DataFrame(dge_C)
robjects.globalenv['dge_N'] = robjects.DataFrame(dge_N)

In [ ]:
%%R
save.image("topgo_input.Rdata")

In [ ]:
%%R
library(topGO)
gene_names = rownames(full_with_go)
cambium_interesting = rownames(dge_C)
needle_interesting = rownames(dge_N)
gene_id_2go  = readMappings(file="go_mappings.txt")
interesting = list()
interesting$cambium = cambium_interesting
interesting$needle = needle_interesting
godata = list()
gentables = list()
gentables_bh = list()
gentables_qval = list()
onts = c("BP","CC", "MF")
sigs = list()
descriptions = list()
for (i in 1:length(onts)) {
    for (j in 1:length(interesting)) {
        interest = interesting[[j]]
        gene_list <- factor(as.integer(gene_names %in% interest))
        names(gene_list) <- gene_names
        description=paste(names(interesting)[j], onts[i], sep="-")
        descriptions = append(descriptions, description)
        GOdata = new("topGOdata",
                     description=description,
                     ontology = onts[i], 
                     allGenes = gene_list, 
                     annot = annFUN.gene2GO, 
                     gene2GO = gene_id_2go,
                     nodeSize=2)
        print(GOdata)
        godata = append(godata, GOdata)
        classicFisher = runTest(GOdata, algorithm = "classic", statistic = "fisher")
        weight01Fisher = runTest(GOdata, algorithm = "weight01", statistic = "fisher")
        sigs = append(sigs, classicFisher)
        printGraph(GOdata, 
                   classicFisher, 
                   firstSigNodes = 2, 
                   fn.prefix = paste("tGO", "for", description(GOdata)), 
                   #fn.prefix = paste("tGOslim", "for", description(GOdata)), 
                   useInfo = "all")
        
        gt = GenTable(GOdata, 
                      classicFisher=classicFisher, 
                      weight01Fisher=weight01Fisher, 
                      topNodes=length(classicFisher@score), 
                      orderBy="classicFisher", numChar=1000)
        gentables = append(gentables, list(gt))
        
        fisher_p = as.numeric(gt[,"classicFisher"])
        
        gt.bh = gt[which(p.adjust(fisher_p,method="BH")<=0.05),]
        
        #print(qvalue(fisher_p))
    
        
        gt.qval = gt[which(qvalue(fisher_p)$qvalues<=0.05),]
        gentables_bh = append(gentables_bh, list(gt.bh))
        gentables_qval = append(gentables_qval, list(gt.qval))
        write.table(gt, file=paste(description(GOdata), ".txt", sep=""), row.names=F)
        write.table(gt.bh, file=paste(description(GOdata), "_bh.txt", sep=""), row.names=F)
        write.table(gt.qval, file=paste(description(GOdata), "_qval.txt", sep=""), row.names=F)  
    }
}
save.image("topgo.Rdata")

In [ ]:
%%R
load("topgo.Rdata")

In [ ]:
%%R
ls()

In [ ]:
gentables = {}
for i, desc in enumerate(r("descriptions")):
    d = {"gt":None, "gt_bh":None}
    d['gt'] = ri2py(r("gentables")[i])
    d['gt_bh'] = ri2py(r("gentables_bh")[i])
    gentables[desc[0]] = d

In [ ]:
gentables.keys()

In [ ]:
for key in gentables:
    print key
    gt = gentables[key]['gt']
    bh = gentables[key]['gt_bh']
    gt = gt.convert_objects(convert_numeric=True)
    bh = bh.convert_objects(convert_numeric=True)
    gt_sig = gt[gt.classicFisher<0.05]
    bh_sig = bh[bh.classicFisher<0.05]
    if len(gt_sig) > 0:
        print "raw: %d" % len(gt_sig)
    if len(bh_sig) > 0:
        print "bh: %d" % len(bh_sig)

In [ ]:
dill.dump(gentables, open("gentables.dill", "w"))

In [ ]:
dge_C.to_csv("dge_cambium.txt", sep="\t")
dge_N.to_csv("dge_needle.txt", sep="\t")

##Singletons, etc

In [ ]:
counts = pd.read_csv("all.fa.clean.sequin.trim_output/contig_member.counts", sep="\t",header=0, index_col=0)

In [ ]:
len(counts[counts.apply(np.sum, axis=1)==1])

In [ ]:
needle_unigenes = combined[combined.N>0]
cambium_unigenes = combined[combined.C>0]

In [ ]:
from matplotlib_venn import venn2

In [ ]:
sns.set_context("talk")
venn2([set(needle_unigenes.index), set(cambium_unigenes.index)],
     set_labels = ['Needle', 'Cambium'])
plt.show()

##Read length distribution

In [ ]:
read_lengths = {}
for read in SeqIO.parse("all.fa.clean.sequin.trim", "fasta"):
    sample = read.id.split("_")[0]
    if not sample in read_lengths:
        read_lengths[sample] = []
    read_lengths[sample].append(len(read))

In [ ]:
c_lens = []
n_lens = []
for r, d in read_lengths.items():
    print r, len(d), pd.Series(d).describe()
    robjects.globalenv[r] = d
    if "C" in r:
        c_lens.extend(d)
    else:
        n_lens.extend(d)

In [ ]:
%R -i c_lens -i n_lens

In [ ]:
%%R
t.test(c_lens, n_lens)

##PCA

In [ ]:
counts.head()

In [ ]:
def get_tissue(row):
    if row.C > 0 and row.N > 0:
        return "Both"
    elif row.C > 0:
        return "Cambium"
    elif row.N > 0:
        return "Needle"

In [ ]:
count_df = pd.DataFrame(counts)

In [ ]:
count_df['C'] = count_df.apply(lambda x: x.P32C + x.P40C, axis=1)
count_df['N'] = count_df.apply(lambda x: x.P32N + x.P40N, axis=1)

In [ ]:
count_df.head()

In [ ]:
count_df['tissue'] = count_df.apply(get_tissue, axis=1)
count_df['total'] = count_df.apply(lambda row: row.C + row.N, axis=1)

def get_color(row):
    if row.tissue == "Needle":
        return "green"
    elif row.tissue == "Cambium":
        return "blue"
    else:
        return "gray"

count_df['color'] = count_df.apply(get_color, axis=1)

In [ ]:
count_df.head()

In [ ]:
%R -i count_df

In [ ]:
%%R
len_pca = prcomp(count_df[,1:4], center=T, scale=T)
print(len_pca)
plot(len_pca, type="l")
print(summary(len_pca))

In [ ]:
%%R
library(ggbiplot)

In [ ]:
%%R
g <- ggbiplot(len_pca, 
              obs.scale = 1, 
              var.scale = 1,
              ellipse = T, 
              circle = T,
             groups=count_df[,7],size=count_df[,8])
print(g)
pdf("count_pca.pdf")
print(g)
dev.off()

In [ ]:
%%R
count_df[,9] = as.character(count_df[,9])
plot(len_pca$x[,1], len_pca$x[,2], pch="", main="PCA of unigene expression (n=1343) for 4 samples", xlab="PC1 (43.2%)", ylab="PC2 (34.7%)")
points(len_pca$x[,1], len_pca$x[,2], pch=21, bg=alpha(count_df[,9], 0.6), , cex=2)
legend(28,-23,legend=c("Needle", "Cambium", "Both"), col=c("green", "blue", "gray"), pch=20, pt.cex=2.5)

In [ ]:
%%R
length(rownames(count_df))

##Blast against gene contigs in *P. abies*

See `abies_blast.ipynb`